In [6]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


In [7]:
# minsearch repository https://github.com/alexeygrigorev/minsearch/tree/main

In [15]:
#implementing the search engine 
import minsearch

In [16]:
# Getting the indexed FAQ documents

import json

In [17]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [18]:
# restructuring the json to contain document records for each course
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [19]:
#indexing the document so we can search through it 

index = minsearch.Index(
    text_fields = ['question','text','section'],
    keyword_fields=['course']
)

In [20]:
index.fit(documents)

In [21]:
# texting the index 
boost = {'question': 3.0, 'section':0.5} #this means the question field is 3 times more important than the text field


results = index.search(
    query='the courese has already started, can I still enroll?',
    boost_dict =boost,
    filter_dict = {'course':'data-engineering-zoomcamp'},
    num_results=5
)

In [22]:
results

## PASSING THE CONTEXT TO AN LLM 

In [1]:
import os 
#os.environ['OPENAI_API_KEY']

In [24]:
from openai import OpenAI

In [27]:
client = OpenAI()
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":'user', "content":'the courese has already started, can I still enroll?'}])

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4o` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
response.choices[0].message.content

In [28]:
#Setting the context

prompt_template = """
You are course teaching assistant. Answer the QUESTION based on context from the FAQ databse.
Use only the facts from the CONTEXT when answering the QUESTION 
If the CONTEXT doesn't contain the answer , output NONE


QUESTION: {question}

CONTEXT: {context}

"""

In [29]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [31]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Environment - Is Python 3.9 still the recommended version to use in 2024?
answer: Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]
But Python 3.10 and 3.11 should work fine.

section: General course-related questions
question: How can we contribute to the course?
answer: Star the repo! Share it with friends if you find it useful ❣️
Create a PR if you see you can improve the text or the structure of the repository.

section: General course-related questions
question: Are we still using the NYC Trip data for January 2021? Or are we using the 2022 data?
answer: We will u

In [ ]:
response.choices[0].message.content

### Modularizing the code 

In [ ]:
def search(query):
    